In [1]:
from sqlalchemy import Table, Column, Integer, String, ForeignKey, create_engine, Float
from sqlalchemy.orm import relationship, declarative_base, sessionmaker

Base = declarative_base()

# Association table for many-to-many relationship between Path and Environment
path_env_association = Table(
    'path_env_association',
    Base.metadata,
    Column('path_id', ForeignKey('paths.id'), primary_key=True),
    Column('env_id', ForeignKey('environments.id'), primary_key=True)
)

class Journey(Base):
    __tablename__ = 'journeys'
    id = Column(Integer, primary_key=True)
    name = Column(String)

    # One-to-many relationship: Journey -> Paths
    paths = relationship("Path", back_populates="journey")

class Path(Base):
    __tablename__ = 'paths'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    journey_id = Column(Integer, ForeignKey('journeys.id'))

    # Back reference to Journey
    journey = relationship("Journey", back_populates="paths")

    # Many-to-many relationship with Environment
    envs = relationship("Environment", secondary=path_env_association, back_populates="paths")

class Environment(Base):
    __tablename__ = 'environments'
    id = Column(Integer, primary_key=True)
    name = Column(String)  # e.g., "Hot_Env"
    value = Column(String) # e.g., "50°C"

    # Many-to-many back reference
    paths = relationship("Path", secondary=path_env_association, back_populates="envs")
    int_parameters = relationship("IntParameter", back_populates="environment", cascade="all, delete-orphan")
    float_parameters = relationship("FloatParameter", back_populates="environment", cascade="all, delete-orphan")
    string_parameters = relationship("StringParameter", back_populates="environment", cascade="all, delete-orphan")

    
class StringParameter(Base):
    __tablename__ = 'string_parameters'
    id = Column(Integer, primary_key=True)
    name = Column(String)   # e.g., "Temperature"
    value = Column(String)  # e.g., "50°C"
    environment_id = Column(Integer, ForeignKey('environments.id'))

    # Back reference to Environment
    environment = relationship("Environment", back_populates="parameters")

class IntParameter(Base):
    __tablename__ = 'int_parameters'
    id = Column(Integer, primary_key=True)
    name = Column(String)   # e.g., "Temperature"
    value = Column(Integer)  # e.g., "50°C"
    environment_id = Column(Integer, ForeignKey('environments.id'))

    # Back reference to Environment
    environment = relationship("Environment", back_populates="int_parameters")

class FloatParameter(Base):
    __tablename__ = 'float_parameters'
    id = Column(Integer, primary_key=True)
    name = Column(String)   # e.g., "Temperature"
    value = Column(Float)  # e.g., "50°C"
    environment_id = Column(Integer, ForeignKey('environments.id'))

    # Back reference to Environment
    environment = relationship("Environment", back_populates="float_parameters")

In [2]:
# Create an in-memory SQLite DB for demo
engine = create_engine('sqlite:///:memory:', echo=True)
Base.metadata.create_all(engine)

Session = sessionmaker(bind=engine)
session = Session()


2025-08-25 10:10:29,454 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-08-25 10:10:29,454 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("path_env_association")
2025-08-25 10:10:29,454 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-08-25 10:10:29,455 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("path_env_association")
2025-08-25 10:10:29,455 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-08-25 10:10:29,456 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("journeys")
2025-08-25 10:10:29,456 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-08-25 10:10:29,457 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("journeys")
2025-08-25 10:10:29,457 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-08-25 10:10:29,458 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("paths")
2025-08-25 10:10:29,458 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-08-25 10:10:29,458 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("paths")
2025-08-25 10:10:29,459 INFO sqlalchemy.en

In [3]:
# Create environments
hot_env = Environment(name="Hot_Env", value="50")
cold_env = Environment(name="Cold_Env", value="-10")

# Create journeys
journey1 = Journey(name="Safari Adventure")
journey2 = Journey(name="Mountain Trek")

# Create paths
forest_path = Path(name="Forest", journey=journey1, envs=[hot_env])
desert_path = Path(name="Desert", journey=journey1, envs=[hot_env])
snow_path = Path(name="Snow", journey=journey2, envs=[cold_env])

# Add and commit
session.add_all([journey1, journey2, forest_path, desert_path, snow_path])
session.commit()


2025-08-25 10:10:30,225 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-08-25 10:10:30,227 INFO sqlalchemy.engine.Engine INSERT INTO journeys (name) VALUES (?) RETURNING id
2025-08-25 10:10:30,227 INFO sqlalchemy.engine.Engine [generated in 0.00006s (insertmanyvalues) 1/2 (ordered; batch not supported)] ('Safari Adventure',)
2025-08-25 10:10:30,227 INFO sqlalchemy.engine.Engine INSERT INTO journeys (name) VALUES (?) RETURNING id
2025-08-25 10:10:30,228 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/2 (ordered; batch not supported)] ('Mountain Trek',)
2025-08-25 10:10:30,230 INFO sqlalchemy.engine.Engine INSERT INTO environments (name, value) VALUES (?, ?) RETURNING id
2025-08-25 10:10:30,230 INFO sqlalchemy.engine.Engine [generated in 0.00005s (insertmanyvalues) 1/2 (ordered; batch not supported)] ('Hot_Env', '50')
2025-08-25 10:10:30,230 INFO sqlalchemy.engine.Engine INSERT INTO environments (name, value) VALUES (?, ?) RETURNING id
2025-08-25 10:10:30,231 INFO sqlalchemy.engine

In [4]:
from sqlalchemy import and_

target_value = "50"

results = (
    session.query(Path)
    .join(Path.envs)   # Join with Environment via association table
    .filter(
        Path.name.in_(["Forest", "Desert"]),
        Environment.name == "Hot_Env",
        # Environment.value == target_value
    )
    .all()
)

for path in results:
    print(f"Path: {path.name}, Journey: {path.journey.name}")


2025-08-25 10:10:30,722 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-08-25 10:10:30,724 INFO sqlalchemy.engine.Engine SELECT paths.id AS paths_id, paths.name AS paths_name, paths.journey_id AS paths_journey_id 
FROM paths JOIN path_env_association AS path_env_association_1 ON paths.id = path_env_association_1.path_id JOIN environments ON environments.id = path_env_association_1.env_id 
WHERE paths.name IN (?, ?) AND environments.name = ?
2025-08-25 10:10:30,725 INFO sqlalchemy.engine.Engine [generated in 0.00033s] ('Forest', 'Desert', 'Hot_Env')
2025-08-25 10:10:30,726 INFO sqlalchemy.engine.Engine SELECT journeys.id AS journeys_id, journeys.name AS journeys_name 
FROM journeys 
WHERE journeys.id = ?
2025-08-25 10:10:30,727 INFO sqlalchemy.engine.Engine [generated in 0.00039s] (1,)
Path: Forest, Journey: Safari Adventure
Path: Desert, Journey: Safari Adventure


In [ ]:
results = (
    session.query(Path)
    .join(Path.envs)
    .join(Path.journey)  # Join Journey
    .filter(
        Journey.name == "Safari Adventure",
        Path.name.in_(["Forest", "Desert"]),
        Environment.name == "Hot_Env",
        Environment.value == target_value
    )
    .all()
)
for result in results:
    print(f"Journey: {result.journey.name}")
    print(f"Path: {result.name}")
    for env in result.envs:
        print(f"  {env.name}: {env.value}")
    print()

2025-08-25 10:10:31,225 INFO sqlalchemy.engine.Engine SELECT paths.id AS paths_id, paths.name AS paths_name, paths.journey_id AS paths_journey_id 
FROM paths JOIN path_env_association AS path_env_association_1 ON paths.id = path_env_association_1.path_id JOIN environments ON environments.id = path_env_association_1.env_id JOIN journeys ON journeys.id = paths.journey_id 
WHERE journeys.name = ? AND paths.name IN (?, ?) AND environments.name = ? AND environments.value = ?
2025-08-25 10:10:31,225 INFO sqlalchemy.engine.Engine [generated in 0.00063s] ('Safari Adventure', 'Forest', 'Desert', 'Hot_Env', '50')
Journey: Safari Adventure
Path: Forest
2025-08-25 10:10:31,227 INFO sqlalchemy.engine.Engine SELECT environments.id AS environments_id, environments.name AS environments_name, environments.value AS environments_value 
FROM environments, path_env_association 
WHERE ? = path_env_association.path_id AND environments.id = path_env_association.env_id
2025-08-25 10:10:31,227 INFO sqlalchemy.e